In [ ]:
!wget https://github.com/git-lfs/git-lfs/releases/download/v3.4.0/git-lfs-linux-amd64-v3.4.0.tar.gz
!tar -xvzf git-lfs-linux-amd64-v3.4.0.tar.gz
!PREFIX=/opt/app-root/src/.local ./git-lfs-3.4.0/install.sh

In [ ]:
!git clone https://github.com/caikit/caikit-nlp.git

In [ ]:
!pip install --upgrade pip
!pip install ./caikit-nlp

In [ ]:
!git clone https://github.com/opendatahub-io/caikit-tgis-serving.git

In [ ]:
!cp caikit-tgis-serving/utils/convert.py .

In [ ]:
!pip install --upgrade huggingface_hub

In [ ]:
hftokensecret = %env hf_token
!huggingface-cli login --token $hftokensecret

In [ ]:
!git clone https://huggingface.co/neoxu1688/Llama-2-7b-chat-hf-fine-tuned

In [ ]:
!./convert.py --model-path ./Llama-2-7b-chat-hf-fine-tuned --model-save-path ./Llama-2-7b-chat-hf-fine-tuned-caikit

# Move model to S3

In [ ]:
! pip install --upgrade pip
! pip install boto3 botocore

In [ ]:
import os
import boto3
import botocore

In [ ]:
%env AWS_ACCESS_KEY_ID = minio
%env AWS_SECRET = minio123
%env AWS_REGION = us-east-1
%env AWS_BUCKET_NAME = my-ai-bucket

In [ ]:
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

In [ ]:
session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                 aws_secret_access_key=aws_secret_access_key)
 
s3_resource = session.resource(
    's3',
     config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)
 
bucket = s3_resource.Bucket(bucket_name)

In [ ]:
def upload_directory_to_s3(local_directory, s3_prefix):
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)

In [ ]:
def list_objects(prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)

In [ ]:
upload_directory_to_s3("Llama-2-7b-chat-hf-fine-tuned-caikit", f"model/Llama-2-7b-chat-hf-fine-tuned")

In [ ]:
list_objects(f"model/Llama-2-7b-chat-hf-fine-tuned")